# T-SNE Plot to Show the Distribution Shift From Ego4D to EPIC-KITCHENS

Draw a T-SNE plot for randomly selected videos from Ego4D and EPIC-KITCHENS.

In [ ]:
import glob
import random

import numpy as np
import pandas as pd
import torch

random.seed(42)

ego4d_emb_files = glob.glob("../../eilev-blip2-opt-2.7b-vision-model-embs/ego4d/*.pt")
random.shuffle(ego4d_emb_files)
ego4d_emb_files = ego4d_emb_files[:20000]
print("Ego4D")
for file in ego4d_emb_files[:3]:
    print(file)

epic_kitchens_emb_files = glob.glob(
    "../../eilev-blip2-opt-2.7b-vision-model-embs/EPIC-KITCHENS/*.pt"
)
random.shuffle(epic_kitchens_emb_files)
epic_kitchens_emb_files = epic_kitchens_emb_files[:20000]
print("EPIC-KITCHENS")
for file in epic_kitchens_emb_files[:3]:
    print(file)

vid_embs = torch.stack(
    [torch.load(file) for file in ego4d_emb_files + epic_kitchens_emb_files]
).numpy()
print(f"vid_embs.shape = {vid_embs.shape}")
# 0 = ego4d, 1 = epic-kitchens
y = np.array([0] * len(ego4d_emb_files) + [1] * len(epic_kitchens_emb_files))
assert vid_embs.shape[0] == y.shape[0]

sample_df = pd.DataFrame()
sample_df["y"] = y
sample_df["Dataset"] = sample_df["y"].apply(
    lambda i: "Ego4D" if i == 0 else "EPIC-KITCHENS"
)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


def tsne(vid_embs: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    pca = PCA(n_components=50)
    pca_result = pca.fit_transform(vid_embs)

    tsne = TSNE()
    tsne_results = tsne.fit_transform(pca_result)
    return tsne_results[:, 0], tsne_results[:, 1]


tsne_one, tsne_two = tsne(vid_embs)
sample_df["tsne-one"] = tsne_one
sample_df["tsne-two"] = tsne_two

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")
plt.rcParams["font.family"] = "stixgeneral"
plt.figure(figsize=(16, 10))
sns.scatterplot(
    x="tsne-one",
    y="tsne-two",
    hue="Dataset",
    palette=sns.color_palette("hls", 2),
    data=sample_df,
    legend="full",
)
plt.xlabel(None)
plt.ylabel(None)
plt.tight_layout()
plt.show()

Figure out which common actions from EPIC-KITCHENS are present in Ego4D.

In [ ]:
import csv
from collections import Counter


def count_actions(*files):
    counter = Counter()
    for file in files:
        with open(file, newline="") as f:
            csvreader = csv.DictReader(f)
            for item in csvreader:
                if not item["structured_verb"] or not item["structured_noun"]:
                    continue
                counter[(item["structured_verb"], item["structured_noun"])] += 1
    return counter


ego4d_annotation_files = [
    "../../ego4d/fho_main_train_frames-448px-subsample-8/narrated_actions.csv",
    "../../ego4d/fho_main_val_frames-448px-subsample-8/narrated_actions.csv",
    "../../ego4d/fho_main_test_frames-448px-subsample-8/narrated_actions.csv",
]
ego4d_action_counter = count_actions(*ego4d_annotation_files)

epic_kitchens_annotation_files = [
    "../../EPIC-KITCHENS/train-epic-kitchens-subsample-8/narrated_actions.csv",
    "../../EPIC-KITCHENS/val-epic-kitchens-subsample-8/narrated_actions.csv",
]
epic_kitchens_action_counter = count_actions(*epic_kitchens_annotation_files)

print("Ego4D")
for action in ego4d_action_counter.most_common(10):
    print(action)
print("EPIC-KITCHENS")
for action in epic_kitchens_action_counter.most_common(10):
    print(action)

In [ ]:
print(
    "('turn_on_(switch_on,_start,_light)', 'faucet_(faucet,_tap)'): "
    + str(
        ego4d_action_counter[
            ("turn_on_(switch_on,_start,_light)", "faucet_(faucet,_tap)")
        ]
    )
)
print(
    "('turn_off_(turn_off,_switch_off)', 'faucet_(faucet,_tap)'): "
    + str(
        ego4d_action_counter[
            ("turn_off_(turn_off,_switch_off)", "faucet_(faucet,_tap)")
        ]
    )
)
print(
    "('open', 'cabinet_(cabinet,_compartment,_cupboard)'): "
    + str(ego4d_action_counter[("open", "cabinet_(cabinet,_compartment,_cupboard)")])
)
print("('open', 'drawer'): " + str(ego4d_action_counter[("open", "drawer")]))
print(
    "('close', 'cabinet_(cabinet,_compartment,_cupboard)'): "
    + str(ego4d_action_counter[("close", "cabinet_(cabinet,_compartment,_cupboard)")])
)

Draw a T-SNE plot for these actions.

In [ ]:
from collections import defaultdict
from pathlib import Path


def get_action_annotations_map(
    annotation_files: list[str],
) -> dict[tuple[str, str], list[dict]]:
    action_annotations_map: dict[tuple[str, str], list[dict]] = defaultdict(list)
    for file in annotation_files:
        with open(file, newline="") as f:
            csvreader = csv.DictReader(f)
            for item in csvreader:
                if not item["structured_verb"] or not item["structured_noun"]:
                    continue
                action_annotations_map[
                    (item["structured_verb"], item["structured_noun"])
                ].append(item)

    return action_annotations_map


def read_embs(
    action: tuple[str, str],
    action_annotations_map: dict[tuple[str, str], list[dict]],
    emb_dir: str,
) -> np.ndarray:
    vid_emb_list: list[torch.Tensor] = []
    for item in action_annotations_map[action]:
        vid_emb_list.append(torch.load(Path(emb_dir) / (item["frame_path"] + ".pt")))

    return torch.stack(vid_emb_list).numpy()


ego4d_actions = [
    ("turn_on_(switch_on,_start,_light)", "faucet_(faucet,_tap)"),
    ("turn_off_(turn_off,_switch_off)", "faucet_(faucet,_tap)"),
    ("open", "cabinet_(cabinet,_compartment,_cupboard)"),
]
epic_kitchens_actions = [("turn-on", "tap"), ("turn-off", "tap"), ("open", "cupboard")]

ego4d_action_annotations_map = get_action_annotations_map(ego4d_annotation_files)
epic_kitchens_action_annotations_map = get_action_annotations_map(
    epic_kitchens_annotation_files
)

action_dfs: list[pd.DataFrame] = []

for ego4d_action, epic_kitchens_action in zip(ego4d_actions, epic_kitchens_actions):
    ego4d_embs = read_embs(
        ego4d_action,
        ego4d_action_annotations_map,
        "../../eilev-blip2-opt-2.7b-vision-model-embs/ego4d/",
    )
    epic_kitchens_embs = read_embs(
        epic_kitchens_action,
        epic_kitchens_action_annotations_map,
        "../../eilev-blip2-opt-2.7b-vision-model-embs/EPIC-KITCHENS/",
    )
    vid_embs = np.concatenate((ego4d_embs, epic_kitchens_embs), axis=0)
    tsne_one, tsne_two = tsne(vid_embs)

    df = pd.DataFrame()
    df["y"] = np.array([0] * ego4d_embs.shape[0] + [1] * epic_kitchens_embs.shape[0])
    df["Dataset"] = df["y"].apply(lambda i: "Ego4D" if i == 0 else "EPIC-KITCHENS-100")
    df["tsne-one"] = tsne_one
    df["tsne-two"] = tsne_two
    action_dfs.append(df)

In [ ]:
sns.set(style="darkgrid")
plt.rcParams["font.family"] = "stixgeneral"

fig, axs = plt.subplots(1, 4, figsize=(16, 4))

ax = axs[0]
sns.scatterplot(
    x="tsne-one",
    y="tsne-two",
    hue="Dataset",
    palette=sns.color_palette("hls", 2),
    data=sample_df,
    ax=ax,
)
ax.set_xlabel(None)
ax.set_ylabel(None)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_title("Random 40K Subset", fontsize=30, fontweight="bold")
ax.get_legend().remove()
ax.collections[0].set_rasterized(True)
ax.grid(False)

for i, ((verb, noun), df) in enumerate(zip(epic_kitchens_actions, action_dfs)):
    ax = axs[i + 1]
    sns.scatterplot(
        x="tsne-one",
        y="tsne-two",
        hue="Dataset",
        palette=sns.color_palette("hls", 2),
        data=df,
        ax=ax,
    )
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title(
        (" ".join(verb.split("-") + noun.split("-"))).title(),
        fontsize=30,
        fontweight="bold",
    )
    ax.get_legend().remove()
    ax.collections[0].set_rasterized(True)
    ax.grid(False)

handles, labels = ax.get_legend_handles_labels()

plt.tight_layout()
plt.savefig("tsne.pdf", bbox_inches="tight")
plt.show()